# Using cURL to work with the Db2 Console

This Jupyter Notebook contains examples of how to setup and manage the Db2 Data Management Console using cURL. 

cURL is a command-line tool for getting or sending data, including files, using URL syntax. The name stands for "Client URL". It is particularly useful to write simple scripts to access the RESTful APIs of the Db2 Console. In this example we include cURL calls in a BASH script. The script constructs the URLs that contain the RESTful calls, then submits them to the Db2 Console's communication port. If you don't have access to a Jupyter Notebook environment, CURL is a simple way to get started with the Db2 Console APIs.

Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

This hands on lab will be calling the Db2 Data Management Console as a service. However you can explore it through the user interface as well. Just click on the following link to try out the console that is already and setup in this lab: http://localhost:11080/console. If you have not already logged in you can use the following:
* Userid: db2inst1
* Password: db2inst1

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

### Prepare the notebook
Click on the next cell and press Shift-Return to refresh all of the examples in this notebook. This sets up the notebook example code.

In [1]:
%run refresh.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### How to Copy Code and Examples
Throughout this lab there are code samples that need to be copied and modified in a text editor. Any commands that need to be executed from a command line are found in grey boxes (an example is found below) has been designed to be easily copied.
1. Click on the next cell

In [2]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
Sample commands are found in cells like this.
</pre>
</div>

The entire contents of the text in the cell will be automatically copied when you click on the cell. The color of the background will change color briefly to indicate that the copy has completed. To paste commands into a terminal window, use the key combination Control-Shift-v.

It may be easier to keep a terminal window on top of the Jupyter notebook when running these commands. When you have a terminal window displayed, right click on the title bar and select Always on Top to keep the screen visible during the duration of the lab.


## Constructing API calls
To access the Db2 Console service, first authenticate with the service and create a reusable token that is used for each call to the service. The token ensures that we don't have to provide a user ID and password each time we run a command while keeping the communication secure.

Each request is constructed of several parts. First, you need to identify the URL of the service. For example: <code>http://localhost:11080</code>. In this example, we assume that the console service is running on the same machine as the Jupyter notebook but it can be remote. Port **11080** is typical for running an unsecured connection. (11081 for https). 

You then need to identify the API and the API version. In this case <code>dbapi/v4</code>.

The next part of the call identifies the REST request and the options. For example <code>'/metrics/applications/connections/current/list'</code>. This is followed by more options separated by an <code>&</code>.

So a complete call might look like this:

<code>http://9.30.210.195:11080/dbapi/v4/metrics/applications/connections?end=1579643744759&include_sys=false&limit=100&offset=0&sort=-application_handle&start=1579640144759</code>

In this case, the options are the start and end time, whether to include system generated connections, how to sort the results, and where to start the list. 

Some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what to do if a statement fails.

The full set of APIs are documented as part of the Db2 Consle: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html).

## Simple cURL example
In this example some simple API calls through cURL in a BASH script. The script constructs the URLs that contain the RESTful calls, then submits them to the Db2 Console's 11080 communication port.

1. Click the Files icon at the bottom left of the screen
2. Double-Click **notebooks**
2. Scroll down and Double-Click **SampleCurl.sh**

This opens the Text Editor so you can review the cURL script used in this example. You can also see a copy a few cells down.

## Install the jq library
The jq library makes parsing the JSON returned by API call very easy. To install this library follow the steps below.

1. Click the Terminal Icon in the bottom left of the screen
2. Enter **sudo apt-get install jq** to install the jq library
3. Click https://stedolan.github.io/jq to find out more about his useful library

In [3]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=112 onmousedown="window.clipline(112)" onmouseup="window.reset(112)">
sudo apt-get install jq
</pre>
</div>

## Running the Sample cURL script
You can run the script from the Terminal or from this notebook. 

To run from the Terminal:
1. Return to the Terminal Window from the previous step
2. Enter **cd notebooks**
3. Enter **sh SampleCurl.sh**

To run in this notebook run the next cell (<code>%%bash</code> is included in the first line to pass the command to the BASH interpreter):

In [4]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=113 onmousedown="window.clipline(113)" onmouseup="window.reset(113)">
cd notebooks
sh SampleCurl.sh
</pre>
</div>

In [ ]:
%%bash
#Define Connection Information
HOST='http://localhost:11080'
    USERID='db2inst1'
    PASSWORD='db2inst1'
    CONNECTION='SAMPLE'

#Get reusable TOKEN
    TOKEN=$(curl -s -X POST $HOST/dbapi/v4/auth/tokens \
      -H 'content-type: application/json' -d '{"userid": '$USERID' ,"password":'$PASSWORD'}' | jq -r '.token') 

#Get JSON with details on database connection
JSON=$(curl -s -X GET \
      $HOST'/dbapi/v4/dbprofiles/'$CONNECTION \
      -H 'authorization: Bearer '$TOKEN \
      -H 'content-type: application/json')

    echo $JSON | jq '.'

#Get JSON with list of schemas in the SAMPLE database
JSON=$(curl -s -X GET \
      $HOST'/dbapi/v4/schemas' \
      -H 'authorization: Bearer '$TOKEN \
      -H 'content-type: application/json' \
      -H 'x-db-profile: '$CONNECTION)

    echo $JSON | jq '.'
        echo
        
#Extract the number of schemas in the Sample Database using the jq library
    echo 'Number of Schemas in' $CONNECTION':'
    echo $JSON | jq -r '.count'    

## Combing Db2 Commands with API Calls
The next example creates a Db2 Database and adds that database to the list of Db2 Console connections. This combines Db2 commands along with cURL API commands into the same script. 

### Giving Db2 Command Line Access to DB2POT
Before we can run the sample script using the DB2POT user we need to ensure it can run the Db2 Command Line Processor. By default, only the DB2INST1 user on this Linux system is setup to run the Db2 Command Line processor. (If you want to find out more about this step there is a great BLOG article on this subject from Ember Crooks: [Db2 Basics: Setting up a Command Line to Work with Db2](https://datageek.blog/en/2019/04/30/db2-basics-setting-up-a-command-line-to-work-with-db2))

1. Click the **Files** icon at the bottom left of the screen
2. Click **Other Locations**
3. Click **Computer**
4. Click **home**
5. Click **db2pot**
6. Click the icon at the top right of the **Files** tool with the three horizonal bars
7. Click the checkbox beside **Show Hidden Files**
8. Double Click **.bashrc**
9. Add the followings commands to the bottom of the **.bashrc** file

In [5]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=115 onmousedown="window.clipline(115)" onmouseup="window.reset(115)">
 # Add the following three lines to the .bashrc file for the DB2POT user
 if [ -f /home/db2inst1/sqllib/db2profile ]; then
     . /home/db2inst1/sqllib/db2profile
 fi
</pre>
</div>

10. Click **Save** at the top right of the Editor
11. Close any open **Terminal** window
12. Click the **Terminal** icon at the bottom of the screen
13. Enter **db2**. The Db2 Command Line processor should start.
14. Enter **Connect to SAMPLE**
15. Enter the following select statement.

In [6]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=116 onmousedown="window.clipline(116)" onmouseup="window.reset(116)">
SELECT TABNAME FROM SYSCAT.TABLES
</pre>
</div>

This should return a list of over 700 tables.

16. Enter **quit** to return to the command line

### Review the Create and Catalog Script
You are now ready to run both Db2 and cURL API commands from a BASH script. Lets review a sample script.
1. Click the Files icon at the bottom left of the screen
2. Double-Click notebooks
3. Scroll down and Double-Click **CreateAndCatalog.sh**
This opens the Text Editor so you can review the cURL script used in this example. Notice that the command accepts parameters for both the name of the database you will create as well as the information you need to pass to the Db2 Console through the API.

The first part of the script assigns input parameters to specific variables that are used in the script.

The next step runs a Db2 Command Line command to create the new database using the $DATABASE name parameter

The next step connects to the Db2 Console service and retrieves a connection token that is used in the following step.

Finally the Db2 Console API command is called to add the new Database to the Db2 console list of databases by creating a new database connection profile. In this example the information required to add the connection to the Db2 Console is contained in a JSON string called the API Payload. It includes all the information needed to add the connection. There are few optional parameters included:
1. A userid and password used to connect to the database and monitor performance and availability
2. A userid and password saved for operational tasks like running SQL or exploring database objects
3. An optional comment ussed to describe the connection.
cURL is then used to send the 'db2profiles API request to Db2 Console along with the JSON payload. 

### Run the Create and Catalog Script
Now try running the script from the Terminal

1. Return to the Terminal Window from the previous step
2. Enter **cd notebooks**. Make sure you are in the notebooks directory.
3. Enter the command below:

In [7]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=114 onmousedown="window.clipline(114)" onmouseup="window.reset(114)">
sh CreateAndCatalog.sh TESTDB TESTDB 50000 localhost db2inst1 db2inst1 TestDBComment
</pre>
</div>

4. Return to the Db2 Console in the broswer
5. Click on the Settings Gear icon at the top right of the Db2 Console
6. Select Connection Profile
7. Check for your new database **TESTDB** in the list of databases

The example script created a profile enables monitoring and saves a userid and password you can use for administration and running SQL.

## Next Steps
Try creating your own command line scripts that include API calls to the Db2 Console. You can also cURL to add databases to different Db2 Console monitoring profiles or make changes to existing monitoring profiles. Have a look at the [Managing the Db2 Management Console](http://localhost:8888/notebooks/Db2_Data_Management_Console_Management.ipynb) lesson to find out more about constructing RESTful API calls to configure the Db2 Console. 

#### Credits: IBM 2020, Peter Kohlmann [kohlmann@ca.ibm.com]